In [1]:
import osmnx as ox
from streamlit_functions import get_nodes_with_tags_in_bbox, count_tag_frequency, gdf_data

In [2]:
places = ["Kreuzberg"]
places_gdf = ox.geocode_to_gdf(places)
# bbox = [S, W, N, E]
bounding_boxes = places_gdf.loc[:, ["bbox_south", "bbox_west", "bbox_north", "bbox_east",]]
places_gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((13.36823 52.49334, 13.36917 52.49315...",52.509382,52.482792,13.45293,13.368229,307579131,relation,55765,52.497644,13.411914,"Kreuzberg, Friedrichshain-Kreuzberg, Berlin, G...",boundary,administrative,0.628946


In [3]:
import json
keys = []
nodes = []
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    unique_tags_dict = count_tag_frequency(nodes)
    num_unique_values = {k:len(v) for k, v in unique_tags_dict.items()}
    num_unique_values = {
        k: v
        for k, v in sorted(
            num_unique_values.items(), key=lambda item: item[1], reverse=True
        )
    }

unique_tags_dict_sorted = [unique_tags_dict[k] for k in list(num_unique_values.keys())]

unique_tags_dict

{'highway': ['traffic_signals',
  'crossing',
  'turning_circle',
  'give_way',
  'bus_stop',
  'stop',
  'elevator',
  'street_lamp',
  'speed_camera',
  'traffic_mirror',
  'traffic_sign'],
 'created_by': ['JOSM',
  'YahooApplet 1.0',
  'Potlatch 0.9c',
  'Potlatch 0.10e',
  'Potlatch 0.10f',
  'Merkaartor 0.13',
  'Navigator-Troll',
  'iLOE 1.9'],
 'TMC': ['Point',
  'negative',
  '9.00',
  '32583',
  '31118',
  '32830',
  '31119',
  '26628',
  'positive',
  '27131',
  '27132',
  '32622',
  '31024',
  '27186',
  '27187',
  '27185',
  '32801',
  '41153',
  '41154',
  '41152',
  '32631',
  '32632',
  '32630',
  '26629'],
 'ele': ['34.8',
  '34.7',
  '36.8',
  '35.4',
  '34.3',
  '36.2',
  '36.3',
  '35.8',
  '36.1',
  '36.9',
  '34.9',
  '34.5',
  '35.2',
  '35.5',
  '32.6',
  '36.0',
  '34.4',
  '66',
  '68',
  '46.7',
  '15',
  '20',
  '37.7',
  '35.3',
  '35.0',
  '2',
  '3',
  '1.5',
  '1.6',
  '2.2',
  '4'],
 'button_operated': ['no', 'yes'],
 'crossing': ['traffic_signals',
  'n

In [6]:
def search_dict(d, substring):
    matches = {}
    for key, value in d.items():
        if substring in key:
            matches[key] = value
    return matches

search_dict(unique_tags_dict, "table")

In [7]:
response_json = "{\n  \"place\": \"Monbijoupark\"\n}"

import sys, json
struct = {}


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2822234876.py, line 10)

In [ ]:

dict(zip(places_gdf["display_name"], places_gdf["projected_area"]))

In [ ]:
places_gdf[["projected_area", "area_unit"]] = places_gdf.apply(lambda row: gdf_data(row, places_gdf.crs), axis=1)
places_gdf

In [ ]:
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    keys = list(count_tag_frequency(nodes).keys())
keys

In [ ]:
import utm
from pyproj import CRS
import geopandas as gpd
def gdf_data(gdf):
    """Get the area of a polygon
    This method is not directly callable by the LLM"""
    places_dict = {}
    for index, row in gdf.iterrows():
        print(index)
        utm_zone = utm.latlon_to_zone_number(gdf.loc[[index], "lat"].values[0], gdf.loc[[index], "lon"].values[0])
        south = gdf.loc[[index], "lat"].values[0] < 0
        crs = CRS.from_dict({"proj": "utm", "zone": utm_zone, "south": south})
        epsg_code = crs.to_authority()[1]
        unit = list({ai.unit_name for ai in crs.axis_info})[0]
        gdf_projected = gdf.loc[[index],:].to_crs(epsg_code)
        area = gdf_projected.area.values[0]
        places_dict[row["display_name"]] = {"area":area,
                                            "unit":unit}

    return places_dict

In [ ]:
gdf_data(places_gdf)